# VOCS v1 data structure 

In [1]:
from xopt.vocs import VOCS, ObjectiveEnum, ConstraintEnum

In [2]:
?VOCS

Init signature:
VOCS(
    *,
    variables: Dict[str, types.ConstrainedListValue] = None,
    constraints: Dict[str, types.ConstrainedListValue] = None,
    objectives: Dict[str, xopt.vocs.ObjectiveEnum] = None,
    constants: Dict[str, Any] = None,
    linked_variables: Dict[str, str] = None,
) -> None
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/Code/GitHub/Xopt/xopt/vocs.py
Type:           ModelMetaclass
Subclasses:     


In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables=None)

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': None}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs


True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives['c']  == ObjectiveEnum.MAXIMIZE

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": null}'

In [10]:
vocs.schema()

{'title': 'VOCS',
 'type': 'object',
 'properties': {'variables': {'title': 'Variables',
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'type': 'number'},
    'minItems': 2,
    'maxItems': 2}},
  'constraints': {'title': 'Constraints',
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'anyOf': [{'type': 'number'},
      {'$ref': '#/definitions/ConstraintEnum'}]},
    'minItems': 2,
    'maxItems': 2}},
  'objectives': {'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/ObjectiveEnum'}},
  'constants': {'title': 'Constants', 'type': 'object'},
  'linked_variables': {'title': 'Linked Variables',
   'type': 'object',
   'additionalProperties': {'type': 'string'}}},
 'definitions': {'ConstraintEnum': {'title': 'ConstraintEnum',
   'description': 'An enumeration.',
   'enum': ['LESS_THAN', 'GREATER_THAN'],
   'type': 'string'},
  'ObjectiveEnum': {'title': 'ObjectiveEnum',
   'description': 'An enumeration.',


# Objective Evaluation

In [11]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data['c']  = data['a'] + data['b']
data['d']  = data['a'] - data['b']
data['e']  = data['a']*2 + data['b']*2
data['f']  = data['a']*2 - data['b']*2
data.index = np.arange(len(data)) + 5 # custom index
data


,a,b,g,c,d,e,f
5,651.667038,-0.037769,1234,651.629269,651.704807,1303.258538,1303.409613
6,779.989487,-0.609896,1234,779.379591,780.599383,1558.759182,1561.198765
7,774.322585,0.689245,1234,775.011830,773.633340,1550.023660,1547.266681
8,510.261628,0.485039,1234,510.746667,509.776589,1021.493335,1019.553178
9,485.926718,0.095319,1234,486.022037,485.831399,972.044074,971.662799
10,143.703237,-0.072981,1234,143.630257,143.776218,287.260513,287.552436
11,154.490766,0.211557,1234,154.702323,154.279209,309.404646,308.558417
12,793.835894,-0.580539,1234,793.255355,794.416433,1586.510710,1588.832867
13,498.751919,-0.719754,1234,498.032166,499.471673,996.064332,998.943346
14,812.232960,-0.197208,1234,812.035752,812.430168,1624.071503,1624.860337


In [12]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [13]:
?form_objective_data

Signature: form_objective_data(objectives: Dict, data, prefix='objective_')
Docstring:
Use objective dict and data (dataframe) to generate objective data (dataframe)

Weights are applied to convert all objectives into mimimization form.

Returns a dataframe with the objective data intented to be minimized.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [14]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-651.629269,651.704807
6,-779.379591,780.599383
7,-775.011830,773.633340
8,-510.746667,509.776589
9,-486.022037,485.831399
10,-143.630257,143.776218
11,-154.702323,154.279209
12,-793.255355,794.416433
13,-498.032166,499.471673
14,-812.035752,812.430168


In [15]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-651.629269,651.704807
6,-779.379591,780.599383
7,-775.011830,773.633340
8,-510.746667,509.776589
9,-486.022037,485.831399
10,-143.630257,143.776218
11,-154.702323,154.279209
12,-793.255355,794.416433
13,-498.032166,499.471673
14,-812.035752,812.430168


In [16]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-651.62926881,  651.70480674],
       [-779.37959111,  780.59938263],
       [-775.01182997,  773.63334033],
       [-510.74666737,  509.77658875],
       [-486.02203695,  485.83139926],
       [-143.63025658,  143.77621787],
       [-154.70232301,  154.27920872],
       [-793.25535479,  794.41643348],
       [-498.03216576,  499.47167315],
       [-812.03575161,  812.43016833]])

In [17]:
?form_constraint_data

Signature: form_constraint_data(constraints: Dict, data, prefix='constraint_')
Docstring:
Use constraint dict and data (dataframe) to generate constraint data (dataframe)
A constraint is satisfied if the evaluation is < 0.

Returns a dataframe with the constraint data.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [18]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1301.258538,-1303.409613
6,1556.759182,-1561.198765
7,1548.023660,-1547.266681
8,1019.493335,-1019.553178
9,970.044074,-971.662799
10,285.260513,-287.552436
11,307.404646,-308.558417
12,1584.510710,-1588.832867
13,994.064332,-998.943346
14,1622.071503,-1624.860337


In [19]:
?form_feasibility_data

Signature: form_feasibility_data(constraints: Dict, data, prefix='feasible_')
Docstring:
Use constraint dict and data to identify feasible points in the the dataset.

Returns a dataframe with the feasibility data.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [20]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False
